In [ ]:
!pip install google_play_scraper 
!pip install app_store_scraper

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.0/128.0 kB 18.1 MB/s eta 0:00:00
  Attempting uninstall: chardet
    Found existing installation: chardet 4.0.0
    Uninstalling chardet-4.0.0:
      Successfully uninstalled chardet-4.0.0
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.15
    Uninstalling urllib3-1.26.15:
      Successfully uninstalled urllib3-1.26.15
  Attempting uninstall: idna
    Found existing installation: idna 3.4
    Uninstalling idna-3.4:
      Successfully uninstalled idna-3.4
  Attempting uninstall: 

# New Section

In [ ]:
# from google_play_scraper import app, Sort, reviews_all
# from app_store_scraper import AppStore
import pandas as pd
import numpy as np
import json, os, uuid
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def saveFileToDrive(path,dataframe):

  with open(path, 'w', encoding = 'utf-8-sig') as f:
    dataframe.to_csv(f)
    print("Saving file : "+path)

In [ ]:
def getGooglePlayStoreReviews(country_code):
  g_play_store_reviews = reviews_all(
      "com.barclays.android.barclaysmobilebanking",
      sleep_milliseconds=0, # defaults to 0
      lang='en', # defaults to 'en'
      country=country_code, # defaults to 'us'
      sort=Sort.NEWEST, # defaults to Sort.MOST_RELEVANT
    )
  g_play_store_df = pd.DataFrame(np.array(g_play_store_reviews),columns=['review'])
  g_play_store_df2 = g_play_store_df.join(pd.DataFrame(g_play_store_df.pop('review').tolist()))
  g_play_store_df2.drop(columns={'userImage'},inplace = True)
  g_play_store_df2.rename(columns= {'score': 'rating','userName': 'user_name', 'reviewId': 'review_id', 'content': 'review_description', 'at': 'review_date', 'replyContent': 'developer_response', 'repliedAt': 'developer_response_date', 'thumbsUpCount': 'thumbs_up'},inplace = True)
  g_play_store_df2.insert(loc=0, column='source', value='Google Play')
  g_play_store_df2.insert(loc=3, column='review_title', value=None)
  g_play_store_df2['laguage_code'] = 'en'
  g_play_store_df2['country_code'] = country_code
  g_play_store_df2['review_date'] = pd.to_datetime(g_play_store_df2['review_date'])
  g_play_store_df2 = g_play_store_df2[(g_play_store_df2['review_date'] > "2022-01-31")]
    
  path = '/content/drive/My Drive/BarclaysReviews/g_play_store_'+country_code+'.csv'

  saveFileToDrive(path,g_play_store_df2)


In [ ]:
getGooglePlayStoreReviews('gb')
# getGooglePlayStoreReviews('us')


NameError: ignored

In [ ]:
g_play_store_uk['review_date'] = pd.to_datetime(g_play_store_uk['review_date']).dt.date
g_play_store_us['review_date'] = pd.to_datetime(g_play_store_us['review_date']).dt.date
saveFileToDrive("/content/drive/MyDrive/BarclaysReviews/g_play_store_uk.csv",g_play_store_uk)
saveFileToDrive("/content/drive/MyDrive/BarclaysReviews/g_play_store_us.csv",g_play_store_us)

In [ ]:
g_play_store_uk = pd.read_csv("/content/drive/MyDrive/BarclaysReviews/g_play_store_uk.csv")
g_play_store_us = pd.read_csv("/content/drive/MyDrive/BarclaysReviews/g_play_store_us.csv")
g_play_store_uk_final = g_play_store_uk.drop(columns={'appVersion'})
g_play_store_us_final = g_play_store_us.drop(columns={'appVersion'})
saveFileToDrive("/content/drive/MyDrive/BarclaysReviews/g_play_store_uk_final.csv",g_play_store_uk_final)
saveFileToDrive("/content/drive/MyDrive/BarclaysReviews/g_play_store_us_final.csv",g_play_store_us_final)
g_play_store_merged = pd.concat([g_play_store_uk_final,g_play_store_us_final])
saveFileToDrive("/content/drive/MyDrive/BarclaysReviews/g_play_store_merged.csv",g_play_store_merged)

In [ ]:
def getAppleStoreReviews(country_code):

  app_store_reviews = AppStore(country_code, 'barclays', '536248734')
  app_store_reviews.review()
  app_store_df = pd.DataFrame(np.array(app_store_reviews.reviews),columns=['review'])
  app_store_df2 = app_store_df.join(pd.DataFrame(app_store_df.pop('review').tolist()))
  app_store_df2.drop(columns={'isEdited'},inplace = True)
  app_store_df2.insert(loc=0, column='source', value='App Store')
  app_store_df2['developer_response_date'] = None
  app_store_df2['thumbs_up'] = None
  app_store_df2['laguage_code'] = 'en'
  app_store_df2['country_code'] = country_code
  app_store_df2.insert(loc=1, column='review_id', value=[uuid.uuid4() for _ in range(len(app_store_df2.index))])
  app_store_df2.rename(columns= {'review': 'review_description','userName': 'user_name', 'date': 'review_date','title': 'review_title', 'developerResponse': 'developer_response'},inplace = True)
  app_store_df2 = app_store_df2.where(pd.notnull(app_store_df2), None)
  app_store_df2['review_date'] = pd.to_datetime(app_store_df2['review_date'])
  app_store_df2 = app_store_df2[(app_store_df2['review_date'] > "2022-01-31")]
  path = '/content/drive/My Drive/BarclaysReviews/app_store_'+country_code+'.csv'

  saveFileToDrive(path,app_store_df2)



In [ ]:
getAppleStoreReviews('gb')

In [ ]:
app_store_gb['review_date'] = pd.to_datetime(app_store_gb['review_date']).dt.date
app_store_us['review_date'] = pd.to_datetime(app_store_us['review_date']).dt.date
saveFileToDrive("/content/drive/MyDrive/BarclaysReviews/app_store_gb.csv",app_store_gb)
saveFileToDrive("/content/drive/MyDrive/BarclaysReviews/app_store_us.csv",app_store_us)

In [ ]:
app_store_gb = pd.read_csv("/content/drive/MyDrive/BarclaysReviews/app_store_gb.csv")
app_store_us = pd.read_csv("/content/drive/MyDrive/BarclaysReviews/app_store_us.csv")
app_store_merged = pd.concat([app_store_gb,app_store_us])
saveFileToDrive("/content/drive/MyDrive/BarclaysReviews/app_store_merged.csv",app_store_merged)

In [ ]:
g_play_store_merged = pd.read_csv("/content/drive/MyDrive/BarclaysReviews/g_play_store_merged.csv")
app_store_merged = pd.read_csv("/content/drive/MyDrive/BarclaysReviews/app_store_merged.csv")
final_merged_reviews = pd.concat([g_play_store_uk_final,app_store_gb])
saveFileToDrive("/content/drive/MyDrive/BarclaysReviews/final_merged_reviews.csv",final_merged_reviews)


In [ ]:
final_merged_reviews = pd.read_csv("/content/drive/MyDrive/BarclaysReviews/final_merged_reviews.csv")

In [ ]:
g_play_store_uk_final.shape

In [ ]:
app_store_gb.shape

In [ ]:
final_merged_reviews.shape

In [ ]:
final_merged_reviews.tail()

In [ ]:
pd.to_datetime(app_store_gb['review_date']).dt.date

In [ ]:
g_play_store_uk_final.columns

In [ ]:
app_store_gb.columns

In [ ]:
g_play_store_uk.review_date.nunique()

In [ ]:
g_play_store_us.review_id.nunique()

In [ ]:
app_store_gb.review_date.nunique()

In [ ]:
!pip install -q transformers #Sentiment
from transformers import pipeline

sentiment_analysis = pipeline("sentiment-analysis",model="siebert/sentiment-roberta-large-english")

In [ ]:
final_merged_reviews = pd.read_csv("/content/drive/MyDrive/BarclaysReviews/final_merged_reviews.csv")
final_merged_reviews['sentiment_result']=final_merged_reviews['review_description'].apply(lambda x: sentiment_analysis(x))

final_merged_reviews['sentiment']=final_merged_reviews['sentiment_result'].apply(lambda x: (x[0]['label']))

final_merged_reviews['sentimentScore']=final_merged_reviews['sentiment_result'].apply(lambda x: x[0]['score'])
saveFileToDrive("/content/drive/MyDrive/BarclaysReviews/final_merged_reviews.csv",final_merged_reviews)

In [ ]:
final_merged_reviews.head()

In [ ]:
!pip install pandas
!pip install numpi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.7/48.7 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for numpi: filename=NuMPI-0.3.1-py3-none-any.whl size=55281 sha256=19ffd3d0dfec4c7d19df7f50a4169bbe31084706ea5990299a28db732f850b11
  Stored in directory: /root/.cache/pip/wheels/47/f5/61/44b3f1d8f05c8632d7b981cadeb71487e44e4d467e0ba5f9fa
Successfully built numpi


In [ ]:
!pip install numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
df = pd.DataFrame(dict1)

NameError: ignored

In [ ]:
dict1 = { 
           "name": ['sweekrati','piyush','vivek'],"marks" : [90,80,70] ,"city" : ['pune','pune','kanpur']
         }

import pandas as pd
import numpy as np         

In [ ]:
df = pd.DataFrame(dict1)
df

,name,marks,city
0,sweekrati,90,pune
1,piyush,80,pune
2,vivek,70,kanpur


In [ ]:
dict1

{'name': ['sweekrati', 'piyush', 'vivek'],
 'marks': [90, 80, 70],
 'city': ['pune', 'pune', 'kanpur']}

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Read in CSV file containing scraped data
data = pd.read_csv("scraped_data.csv")

# Count the number of reviews for each star rating
star_counts = data["star_count"].value_counts()

# Create a pie chart showing the distribution of star ratings
plt.pie(star_counts, labels=star_counts.index, autopct="%1.1f%%")
plt.title("Distribution of Star Ratings")
plt.show()